# Solution: Creating the Connected Agent

## Import the libraries

In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import MessageRole, ConnectedAgentTool

## Create the Project Client

In [2]:
# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]  # Ensure the PROJECT_ENDPOINT environment variable is set

# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

## Run Agent helper function

In [3]:
def run_agent(user_input, thread_id, agent_id):
    # Add a message to the thread
    message = project_client.agents.messages.create(
        thread_id=thread_id,  # ID of the thread to which the message belongs
        role="user",  # Role of the message sender
        content=user_input,  # Message content
    )
    print(f"Created message, ID: {message['id']}")

     # Create and process agent run in thread with tools
    run = project_client.agents.runs.create_and_process(thread_id=thread_id, agent_id=agent_id)
    print(f"Run finished with status: {run.status}")

    # Check the status of the run and print the result
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    elif run.status == "completed":
        last_msg = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread_id, role=MessageRole.AGENT)
        if last_msg:
            print(f"Agent Response: {last_msg.text.value}")

## Setup the Connected Agent Tools

In [4]:
# Get the agent by its ID
weather_agent = project_client.agents.get_agent("asst_TWn4ilX0ZBWbcRIIhsQI4jF9")
itinerary_agent = project_client.agents.get_agent("asst_sKhtRu2UTeBVSlEjJvXahEOz")
travel_product_agent = project_client.agents.get_agent("asst_NIP89D33V1fqf3dxAZ6Vtuv9")
restaurant_agent = project_client.agents.get_agent("asst_DL2iuQSceBrvKIPV9G9kW0mF")

weather_connected_agent = ConnectedAgentTool(
        id=weather_agent.id, name="get_weather", description="Searches for information about weather."
    )
itinerary_connected_agent = ConnectedAgentTool(
        id=itinerary_agent.id, name="create_itinerary", description="Generates itineraries for travel."
    )
travel_product_connected_agent = ConnectedAgentTool(
        id=travel_product_agent.id, name="get_travel_products", description="Retrieves travel product information."
    )
restaurant_connected_agent = ConnectedAgentTool(
        id=restaurant_agent.id, name="get_restaurant_recommendations", description="Provides restaurant recommendations."
    )

## Create the Connected Agent

In [5]:
# Create an agent with the file search tool
agent = project_client.agents.create_agent(
    model=os.environ["MODEL_DEPLOYMENT_NAME"],  # Model deployment name
    name="travel-agent",  # Name of the agent
    instructions="""
        You are an agent that provides travel information. 
        Do not answer questions on your own. 
        Your role is to be an orchestrator who will call the appropriate functions provided to you. 
        Each function that you have available is an agent that can accomplish a specific task. 
        
        Here are descriptions of the tools you have available:
            - Weather Agent: An agent that provides weather information.
            - Itinerary Agent: An agent that creates travel itineraries.
            - Travel Product Agent: An agent that retrieves travel product information.
            - Restaurant Agent: An agent that provides restaurant recommendations.

        Use the tools to gather information and provide a comprehensive response to the user.
        If you are unable to answer the question, please respond with "I don't know".
    """,  # Instructions for the agent
    tools=weather_connected_agent.definitions 
    + itinerary_connected_agent.definitions 
    + travel_product_connected_agent.definitions 
    + restaurant_connected_agent.definitions,  # Tools available to the agent
)
print(f"Created agent, ID: {agent.id}")

Created agent, ID: asst_MunJA13PMmTHdia9YTlS70vs


## Create the Thread

In [6]:
# Create a thread for communication
thread = project_client.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_dIE4Df5kedpQJlpVl31RGdTl


## Run the agent

In [7]:
# Example user input for the agent
user_input = "What is the weather in Paris?"  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input

Created message, ID: msg_WIJp0LTP1a9wIzweXLaU5RLx
Run finished with status: completed
Agent Response: The current weather in Paris includes showers and moderate warnings for thunderstorms. The temperature is around 59°F (15°C) with rain expected for the next several hours. The day's highs are projected to be 65°F (18°C) and the lows around 56°F (13°C). There is a significant chance of rain and high humidity at about 90%【3:0†source】【3:1†source】.


In [8]:
# Example user input for the agent
user_input = """Create an itinerary for Paris given this weather information:
Currently, in Paris, it is partly cloudy with a temperature of 68°F (20°C). 
The forecast for later today predicts heavy downpours and frequent lightning with afternoon thundershowers. 
The high temperature will reach 69°F (21°C), with a 100% chance of rain
"""

run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input

Created message, ID: msg_5dVL7FOLOsXX4tJok0HvYank
Run finished with status: completed
Agent Response: Here is an itinerary for Paris, considering the partly cloudy weather in the morning and heavy downpours in the afternoon:

### Morning:
**9:00 AM - 11:00 AM: Explore Montmartre**
- **Sacre-Coeur Basilica:** Visit this stunning basilica located on the highest point in the city.
- **Place du Tertre:** Stroll around this charming square filled with street artists and quaint cafes.

**11:30 AM - 1:30 PM: Champs-Élysées and Arc de Triomphe**
- **Champs-Élysées:** Walk down this famous avenue and enjoy the shops and cafes.
- **Arc de Triomphe:** Climb to the top for a panoramic view of Paris (Note: You may want to check the weather forecast closely as rain may start early).

### Afternoon:
**2:00 PM - 4:00 PM: Louvre Museum**
- Spend a few hours exploring the vast collection of art, including the Mona Lisa, Venus de Milo, and much more. (Pre-book tickets to avoid long lines).

**4:30 PM - 6

In [12]:
# Example user input for the agent
user_input = "What are the features of the UrbanTraveler?"  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input

Created message, ID: msg_SPmiTP2TXJhlChvA6lUtQWv6
Run finished with status: completed
Agent Response: The UrbanTraveler suitcase features the following:

1. **Robust Material**: Made from durable high-impact ABS plastic.
2. **Expandable Design**: Includes an expandable zipper for additional packing space.
3. **360-Degree Spinner Wheels**: Provides effortless maneuverability with four multi-directional spinner wheels.
4. **TSA-Approved Lock**: Built-in TSA-approved combination lock for enhanced security.
5. **Organized Compartments**: Multiple interior and exterior compartments for efficient packing.
6. **Lightweight**: The suitcase weighs only 7 lbs, making it easy to carry and transport.
7. **Ergonomic Handle**: Telescopic handle with an ergonomic grip for comfortable handling.
8. **Water-Resistant**: Exterior is water-resistant to protect belongings from moisture.

**Technical Specs:**
- **Capacity**: 48 liters
- **Material**: ABS Plastic
- **Dimensions**: 23 inches x 15 inches x 9 i

In [13]:
# Example user input for the agent
user_input = "Provide 3 recommendations for Pizza in Paris?"  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id)  # Run the agent with the user input

Created message, ID: msg_K0v8R3O0uDJmIjRc28bH9Y6H
Run finished with status: completed
Agent Response: Here are three highly recommended pizza restaurants in Paris:

1. **Pizza Julia**
   - **Address:** 43 Rue de Charenton, 75012 Paris, France
   - **Reviews:** Famous for its authentic wood-fired pizzas with fresh ingredients.

2. **Popolare**
   - **Address:** 111 Rue Réaumur, 75002 Paris, France
   - **Reviews:** Known for serving large, delicious pizzas at reasonable prices in a lively atmosphere.

3. **Pink Mamma**
   - **Address:** 20bis Rue de Douai, 75009 Paris, France
   - **Reviews:** A trendy spot offering Italian cuisine, including pizza, with a beautiful interior design.

Enjoy your pizza adventure in Paris!


In [14]:
# Example user input for the agent
user_input = """
    Provide me 3 reviews of the Pizzeria Arrivederci in Paris.    
"""  # Example user input
run_agent(user_input, thread_id=thread.id, agent_id=agent.id) 

Created message, ID: msg_rDwAmWAzt8bOQ61Fo0S8K9Xq
Run finished with status: completed
Agent Response: Here are three recent reviews for **Pizzeria Arrivederci** in Paris:

1. **User: sannak926**
   - **Rating: ⭐⭐⭐⭐⭐**
   - **Review:** "Very good menu and very delicious pizzas. Very popular place! Our table booking was right on time and staff was very friendly!"
   - **Trip Type:** Couples
   - **Date of Visit:** July 31, 2025
   - [Read more](https://www.tripadvisor.com/ShowUserReviews-g187147-d12714552-r1018027105-Reviews-Pizzeria_Arrivederci-Paris_Ile_de_France.html?m=66827#review1018027105)

2. **User: F7796YTingridt**
   - **Rating: ⭐⭐⭐⭐⭐**
   - **Review:** "Great food and excellent service. I highly recommend it. Perfect and cozy place, perfect for family and romantic dinners."
   - **Trip Type:** Family
   - **Date of Visit:** July 31, 2025
   - [Read more](https://www.tripadvisor.com/ShowUserReviews-g187147-d12714552-r1018015470-Reviews-Pizzeria_Arrivederci-Paris_Ile_de_France.h